In [1]:
import json
import pandas as pd

from datetime import datetime
from modules.scoring import get_scoring_set, get_features, score, validate
from modules.training import split_data, transform_data

In [2]:
def display_fanta_squad(predictions, teams, index):
    fanta_team = predictions[predictions['fanta_team'] == teams[index]]
    return fanta_team, index+1    

In [3]:
try:
    
    ###############################################################################################
    status = 'Loading config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    with open('config/config.json') as config:
        config = json.load(config)
    status = 'Loading config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Getting scoring set ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    scoring_set = get_scoring_set(config['league_name'])
    scoring_set = get_features(scoring_set, config['league_name'])
    X_raw, metadata = split_data(scoring_set, metadata_columns=['fanta_team', 'date', 'role', 'player_id', 'player'], production=True)
    status = 'Getting scoring set done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################
    
    ###############################################################################################
    status = 'Processing features ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    X = transform_data(X_test_raw=X_raw, categorical_columns=['team', 'opponent', 'is_home', 'position'], production=True)
    status = 'Processing features done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Scoring ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    predictions = score(X, scoring_set, config['league_name'], config['gameweek']['current'])
    status = 'Scoring done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Preparing predictions for display...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    predictions['player'] = predictions['player'].apply(lambda x: x.split(' ')[-1])
    status = 'Preparing predictions for display done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Updating config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    config['gameweek']['current'] = config['gameweek']['current'] + 1
    with open('config/config.json', 'w') as f:
        json.dump(config, f, indent=4)
    status = 'Updating config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

except Exception as e:
    print('Exception: {}'.format(e))
    print('Errored out during: {}'.format(status))

2022-01-24 10:34:54 -> Loading config ...
2022-01-24 10:34:54 -> Loading config done!
2022-01-24 10:34:54 -> Getting scoring set ...
100%|██████████| 174/174 [00:05<00:00, 30.77it/s]2022-01-24 10:38:29 -> Getting scoring set done!
2022-01-24 10:38:29 -> Processing features ...
2022-01-24 10:38:29 -> Processing features done!
2022-01-24 10:38:29 -> Scoring ...
2022-01-24 10:38:29 -> Scoring done!
2022-01-24 10:38:29 -> Preparing predictions for display...
2022-01-24 10:38:29 -> Preparing predictions for display done!
2022-01-24 10:38:29 -> Updating config ...
2022-01-24 10:38:29 -> Updating config done!

Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator OneHotEncoder from version 1.0.2 when using version 1.0.1. This m

In [4]:
_deepnote_run_altair(predictions, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false},"aggregate":"sum"},"y":{"field":"fanta_team","type":"nominal","sort":{"order":"descending","encoding":"x"},"scale":{"type":"linear","zero":true}},"color":{"field":"fanta_team","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [5]:
teams = sorted(list(predictions['fanta_team'].unique()))
index = 0

In [6]:
team, index = display_fanta_squad(predictions, teams, index)

In [7]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [8]:
team, index = display_fanta_squad(predictions, teams, index)

In [9]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [10]:
team, index = display_fanta_squad(predictions, teams, index)

In [11]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [12]:
team, index = display_fanta_squad(predictions, teams, index)

In [13]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [14]:
team, index = display_fanta_squad(predictions, teams, index)

In [15]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [16]:
team, index = display_fanta_squad(predictions, teams, index)

In [17]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [18]:
team, index = display_fanta_squad(predictions, teams, index)

In [19]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [20]:
team, index = display_fanta_squad(predictions, teams, index)

In [21]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [22]:
try:
    
    ###############################################################################################
    status = 'Loading config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    with open('config/config.json') as config:
        config = json.load(config)
    status = 'Loading config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Retrieving past week predictions ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    validation = validate(config['league_name'], config['gameweek']['past'])
    status = 'Retrieving past week predictions done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Preparing predictions for display...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    validation['player'] = validation['player'].apply(lambda x: x.split(' ')[-1])
    status = 'Preparing predictions for display done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Updating config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    config['gameweek']['past'] = config['gameweek']['past'] + 1
    with open('config/config.json', 'w') as f:
        json.dump(config, f, indent=4)
    status = 'Updating config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

except Exception as e:
    print('Exception: {}'.format(e))
    print('Errored out during: {}'.format(status))

2022-01-24 10:38:30 -> Loading config ...
2022-01-24 10:38:30 -> Loading config done!
2022-01-24 10:38:30 -> Retrieving past week predictions ...
100%|██████████| 108/108 [03:03<00:00,  1.70s/it]2022-01-24 10:41:34 -> Retrieving past week predictions done!
2022-01-24 10:41:34 -> Preparing predictions for display...
2022-01-24 10:41:34 -> Preparing predictions for display done!
2022-01-24 10:41:34 -> Updating config ...
2022-01-24 10:41:34 -> Updating config done!



In [23]:
teams = sorted(list(validation['fanta_team'].unique()))
index = 0

In [24]:
team, index = display_fanta_squad(validation, teams, index)

In [25]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [26]:
team, index = display_fanta_squad(validation, teams, index)

In [27]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [28]:
team, index = display_fanta_squad(validation, teams, index)

In [29]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [30]:
team, index = display_fanta_squad(validation, teams, index)

In [31]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [32]:
team, index = display_fanta_squad(validation, teams, index)

In [33]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [34]:
team, index = display_fanta_squad(validation, teams, index)

In [35]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [36]:
team, index = display_fanta_squad(validation, teams, index)

In [37]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [38]:
team, index = display_fanta_squad(validation, teams, index)

In [39]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"error","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"error_type","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
!git add *

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82f6a53d-2812-4762-aa0b-0b8fac2a34e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>